<a href="https://colab.research.google.com/github/PurpleDin0/698S_BOG/blob/master/698S_Project_BOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraper Execution Notebook
BLUF: THis notebook imports the relevant python scraping scripts from github, builds the envirnoment, executes the scripts, saves the data locally, performs basic NLP on the recieved data, then prompts the user to save the resulting information/data (either locally, or to Google Drive).

* [ ] The websites to be searched are located here [Link to website list](https://drive.google.com/file/d/1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU/view?usp=sharing).  Note: website list restricted, for access see the author.
* [ ] The scrapped data should be searched for the terms included in this file [Link to search terms list](https://drive.google.com/file/d/18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2/view?usp=sharing).  Note: file access restricted, for access see the author.

Problem Statement:
Builds a tool to scrape websites and then analyze the information for key terms
and provide the analyst with a rank-ordered list of documents that require human review. 
1. [ ] Construct a set of Python scripts that can be used to scrape a set of [3 websites](https://drive.google.com/file/d/1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU/view?usp=sharing).  
  - [ ] 1.1. Build script to scrape website 1.  
  - [ ] 1.2. Build script to scrape website 2.  
  - [ ] 1.3. Build script to scrape website 3.  
2. [ ] Construct a python Notebook that executes the python script and then performs natural language processing (NLP) to the scrapped data.  
  - [ ] 2.1. Build the initial section that executes the python scripts.
  - [ ] 2.2. Rank order the scrapped results by relevance to the analyist. The ranking will be based on the [list of key terms](https://drive.google.com/file/d/18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2/view?usp=sharing).
  - [ ] 2.3. include an ability to automate the translation of selected
portions of the scraped content.
  - [ ] 2.4. Provide the user with the ability to save the scrapped content.

- Other possible things: 
  -  The code should allow the list of keywords to change over time.  
  -  The system  should immediately flag certain keywords for human review regardless of other content in the document.  
  -  The system should be able to retrieve and analyze office documents (PDFs,
spreadsheets, and word processor documents). 
  - The system should perform sentiment analysis of selected content.


In [1]:
!cp /content/698S_BOG/websites.csv '/content/drive/My Drive/Colab Notebooks/Coursework/698S/Project'

cp: cannot stat '/content/698S_BOG/websites.csv': No such file or directory


# Prepare the environment 
0. git the repo and install all dependancies
1. Install any dependancies
2. Load the list of search sites and search terms

In [1]:
import os
## 0. Clone the repo and then change the working directory to the cloned folder
git_repo =  '698S_BOG'  # Repo we want to clone
git_user = 'PurpleDin0' # User/org we want to clone the repo from
!git clone https://github.com/{git_user}/{git_repo}.git
repo_dir = os.path.join(os.getcwd(), git_repo)
%cd {repo_dir}

## 1. Install the required libraries
!pip install -r requirements.txt

# Caution: Once complete make sure to restart the runtime if required. 

fatal: destination path '698S_BOG' already exists and is not an empty directory.
/content/698S_BOG


In [0]:
## 2. Load the list of search sites and search terms

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# 
# Get the website list file and load it to a dataframe
file_id = '1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('website_list.csv')
website_list_df = pd.read_csv('website_list.csv')
# Get the search term file and load it to a dataframe
file_id = '18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('search_terms.csv')
search_terms_df = pd.read_csv('search_terms.csv')

In [0]:
domain = [s.replace('www.', '') for s in website_list_df.Website.to_list()]
website_list_df['Domain'] = domain

#1. Modify the python scripts to target them at the desired websites
Todolist:
 * [ ] edit the python to change the url to a passed variable the targeted site(s) with the desired targets
 * [ ]  

In [0]:
#TODO write a script that removes the generic sites from the imported python code with the desired target(s)
#TODO 

#2. Run the spider script

In [0]:
ALLOWED_DOMAINS = [s.replace('www.', '') for s in website_list_df.Website.to_list()]
URLS = website_list_df.Website.to_list()

In [9]:
print(URLS)
print(ALLOWED_DOMAINS)

['www.miit.gov.cn', 'www.sastind.gov.cn', 'www.jmjh.miit.gov.cn']
['miit.gov.cn', 'sastind.gov.cn', 'jmjh.miit.gov.cn']


In [0]:
# Standard library imports one per line
import sys
from urllib.parse import urlparse, urljoin

# Third party library imports one per line
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors.lxmlhtml import LxmlLinkExtractor
from scrapy.spiders import Rule
# Global variables

visited = list()
NAME = 'sixnineeight'
"""ALLOWED_DOMAINS = [s.replace('www.', '') for s in website_list_df.Website.to_list()]
URLS = website_list_df.Website.to_list()"""


class SixnineeightSpider(scrapy.Spider):
    """
        Summary:
            `SixnineeightSpider` instantiates the custom spider object.
            Parameters:
                scrapy.Spider (obj):
            Returns:
                (None)
    """
    name = NAME
    allowed_domains = [ALLOWED_DOMAINS]
    start_urls = [URLS]
    custom_settings = {
        'DEPTH_LIMIT': 1  # Gets deep quick...2 is very deep on large sites e.g. 45K+ unique links on CNN. 0=no limit
    }
    rules = (
        Rule(
            LxmlLinkExtractor(allow=()),
            callback='parse_obj',
            follow=True),
    )

    def parse(self, response):
        """
            Summary:
                `parse` method to parse spider responses.
                Parameters:
                    self (obj): self
                    response (obj): scrapy.Response
                Returns:
                    (None)
        """
        links = response.xpath('//a//@href').extract()
        for link in links:
            if "#" in link:
                link = link.split("#")[0]
            if link in visited or urljoin(URLS,
                                          link) in visited or "mailto:" in link or "tel:" in link or "javascript:" in link:
                continue
            else:
                if ALLOWED_DOMAINS in link and link in urlparse(link).netloc:
                    if urlparse(link).scheme == '':
                        link = urljoin(urlparse(URLS).scheme, link)
                    visited.append(link)
                    yield {
                        "link": link
                    }
                    continue
                if urlparse(link).netloc == '' and link not in visited:
                    visited.append(urljoin(URLS, link))
                    yield {
                        "link": urljoin(URLS, link)
                    }
                    continue
                if urlparse(link).netloc != '' and ALLOWED_DOMAINS not in link:
                    continue

        for next_page in links:
            if next_page is not None and urlparse(next_page).netloc == '' or ALLOWED_DOMAINS in next_page:
                if urlparse(next_page).netloc == '':
                    next_page = urljoin(URLS, next_page)
                next_page = response.urljoin(next_page)
                yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)

        pass

In [10]:
process = CrawlerProcess(
        settings={
            # 'FEED_FORMAT': 'pickle',
            # 'FEED_URI': 'file:///***/links.pkl',
            # 'LOG_LEVEL': 'INFO',  # Uncomment if you don't want scrapy to puke DEBUG to the console
            # 'DOWNLOAD_DELAY': 0.25,   # 250 ms of delay, default is random between 0.5 - 1.5
            'TELNETCONSOLE_ENABLED': False,  # On by default...that's dumb ¯\_(ツ)_/¯
            'FEED_FORMAT': 'json',
            'FEED_URI': 'links.json',
            'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
        }
    )
process.crawl(SixnineeightSpider)
try:
    process.start()
    print("It worked!!")
    print("Output {} file written to {}.".format(process.settings['FEED_FORMAT'], process.settings['FEED_URI']))
    sys.exit()
except Exception as e:
    sys.exit("Well, that didn't work... \n {}".format(e))

2020-06-16 01:08:58 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2020-06-16 01:08:58 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
2020-06-16 01:08:58 [scrapy.crawler] INFO: Overridden settings: {'DEPTH_LIMIT': 1, 'FEED_FORMAT': 'json', 'FEED_URI': 'links.json', 'TELNETCONSOLE_ENABLED': False, 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
2020-06-16 01:08:58 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-06-16 01:08:58 [scrapy.middleware] INFO: Enable

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
#Option one for calling the file
#pass the list of sites you want to scrape
%cd {repo_dir}
import sixnineeight

/content/698S_BOG


In [0]:
#Option two for calling the file
%cd /content/698S_BOG
!python3 sixnineeight.py

#import data analysis libraries

In [0]:
import pandas as pd

In [0]:
pd.read_json('links.json')

,link
0,https://www.cnn.com/
1,https://www.cnn.com/us
2,https://www.cnn.com/world
3,https://www.cnn.com/politics
4,https://www.cnn.com/business
...,...
4344,https://www.cnn.com/health/video/2020/03/27/v8...
4345,https://www.cnn.com/travel/video/2020/03/27/v8...
4346,https://www.cnn.com/interactive/call-to-earth
4347,https://www.cnn.com/specials/world/freedom-pro...
